## 不带参数的层（类似激活函数ReLU）

In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module): # 与自定义块相同，都是继承nn.Module并实现init以及forward方法
    def __init__(self):
        super().__init__()

    def forward(self, X):
        # 将X各分量减去其均值
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [2]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

## 带参数的层（类似全连接层nn.Linear）

In [10]:
# 实现自定义的全连接层 + ReLU的合体版
class MyLinear(nn.Module):
    # 需要参数的层初始化需要用户传入：1、输入特征维度 2、输出特征维度 以此计算出W和bias的尺寸
    def __init__(self, in_units, units):
        super().__init__()
        # 初始化完成后创建nn.Parameter的实例
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        # 线性组合 out = XW + b
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear) # 再通过一层relu

In [11]:
linear = MyLinear(5, 3)
linear.weight.data, linear.bias.data

(tensor([[-0.7956,  0.3592, -1.4090],
         [ 0.8709,  1.1065,  1.4657],
         [-0.8068, -0.8250, -1.1996],
         [ 0.8360,  2.0786, -0.8493],
         [ 0.2631, -0.0271, -1.7235]]),
 tensor([ 1.1163, -1.1776,  1.3631]))

In [12]:
linear(torch.rand(2, 5))

tensor([[1.3131, 0.0000, 0.7983],
        [1.3378, 0.4840, 0.0984]])

In [15]:
net = nn.Sequential(MyLinear(64, 8), 
                    MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])